# Closed Systems QFlux Demonstration Notebook

In [ ]:
!pip install qflux

In [ ]:
!apt-get install texlive-latex-base texlive-latex-extra cm-super dvipng

In [ ]:
# Imports
import matplotlib.pyplot as plt
import qutip as qt
import numpy as np
# Jupyter Plotting Preamble...
use_latex = True
# To use Latex:
if use_latex:
    plt.rcParams['text.usetex']      = True
    plt.rcParams['font.family']      = 'Computer Modern Serif'
    plt.rcParams['font.serif']       = ['Computer Modern Roman']
    plt.rcParams['mathtext.bf']      = 'serif:bold'
    plt.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams['mathtext.default'] = 'it'
# To not use Latex:
else:
    plt.rcParams['text.usetex']      = False
    plt.rcParams['font.family']      = 'serif'
    plt.rcParams['font.serif']       = 'Palatino'

# Figure Saving Parameters
plt.rcParams['savefig.dpi']         = 300        # DPI of figure output
plt.rcParams['savefig.transparent'] = True       # Save figure with transparent background
#----------------- Figure Size ------------------#
plt.rcParams['figure.figsize']      = (11.7/1.5, 8.3/1.5)# Figure Size - Adjust as necessary
#------------------ Line Plot -------------------#
plt.rcParams['lines.markersize']    = 4          # Marker size for scatterplots
plt.rcParams['lines.linewidth']     = 1.2        # linewidth
#------------------ Font Size -------------------#
plt.rcParams['font.size']           = 20         # default font size
plt.rcParams['legend.fontsize']     = 20         # Font size for legend
plt.rcParams['axes.labelsize']      = 22         # Font size for tick labels (ticklabels)
plt.rcParams['axes.titlesize']      = 22         # Font size for axis labels (.xlabel/.ylabel)
plt.rcParams['image.aspect']        = 1.0        # Image Aspect Ratio (plt.imshow)
plt.rcParams['image.resample']      = False      # Whether to resample
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']   # Display figures inline as SVG (or 'pdf', 'png', ...)

In [ ]:
import qflux

In [ ]:
from qflux.closed_systems import DynamicsCS, QubitDynamicsCS

## Harmonic Oscillator Dynamics with Qflux [3.1]

In [ ]:
from qflux.closed_systems import DynamicsCS

qho_dyn_obj = DynamicsCS(n_basis=128, xo=1.0, po=0.0, mass=1.0, omega=1.0)
qho_dyn_obj.set_coordinate_operators()
qho_dyn_obj.initialize_operators()
qho_dyn_obj.set_initial_state(wfn_omega=0.2)

total_time = 20.0
N_steps    = 400

qho_dyn_obj.set_propagation_time(total_time, N_steps)
qho_dyn_obj.set_hamiltonian(potential_type='harmonic')
qho_dyn_obj.propagate_qt()
qho_dyn_obj.propagate_SOFT()

In [ ]:
exp_x_qt = qt.expect(qho_dyn_obj.x_op, qho_dyn_obj.dynamics_results_op)
exp_p_qt = qt.expect(qho_dyn_obj.p_op, qho_dyn_obj.dynamics_results_op)

exp_x_ana = [ qho_dyn_obj.xo*np.cos(qho_dyn_obj.omega*t) + (qho_dyn_obj.po/qho_dyn_obj.mass/qho_dyn_obj.omega)*np.sin(qho_dyn_obj.omega*t) for t in qho_dyn_obj.tlist]
exp_p_ana = [ qho_dyn_obj.po*np.cos(qho_dyn_obj.omega*t) -qho_dyn_obj.xo*qho_dyn_obj.omega*qho_dyn_obj.mass*np.sin(qho_dyn_obj.omega*t)  for t in qho_dyn_obj.tlist]

# Plot expectation value, compare to analytic expression
plt.figure(figsize=(9, 6.5))
plt.plot(qho_dyn_obj.tlist, exp_x_qt, label=r'$\left\langle x \right\rangle$ (QuTiP)', color='dodgerblue')
plt.plot(qho_dyn_obj.tlist, exp_x_ana, label=r'$\left\langle x \right\rangle$ (Analytic)',
         lw=0, marker='x', markevery=10, color='dodgerblue', ms=8)
plt.plot(qho_dyn_obj.tlist, exp_p_qt, label=r'$\left\langle p \right\rangle$ (QuTiP)', color='crimson')
plt.plot(qho_dyn_obj.tlist, exp_p_ana, label=r'$\left\langle p \right\rangle$ (Analytic)',
         lw=0, marker='x', markevery=10, color='crimson', ms=8)
plt.ylim(-1.55, 1.70)

plt.legend(ncols=2, loc='upper center')
plt.hlines([-1, 0, 1], min(qho_dyn_obj.tlist), max(qho_dyn_obj.tlist), ls='--', lw=0.85, color='tab:grey', zorder=2)
plt.xlim(min(qho_dyn_obj.tlist), max(qho_dyn_obj.tlist))
plt.xlabel('Time')

## Adenine-Thymine Proton-Transfer Dynamics with Qflux [4.4]

In [ ]:
from qflux.closed_systems import QubitDynamicsCS
from qflux.closed_systems.utils import get_proton_mass, convert_fs_to_au, convert_eV_to_au, convert_au_to_fs
from qiskit_aer import Aer
# Conversion factor
ev2au = convert_eV_to_au(1.0)

# Utility Functions:
def get_doublewell_potential(x, x0=1.9592, f=ev2au, a0=0.0, a1=0.429, a2=-1.126, a3=-0.143, a4=0.563):
    # A-T pair double-well potential in Hartrees (x is in Bohr)
    xi = x/x0
    return f*(a0 + a1*xi + a2*xi**2 + a3*xi**3 + a4*xi**4)


def get_doublewell_potential_second_deriv(x, x0=1.9592, f=ev2au, a0=0.0, a1=0.429, a2=-1.126, a3=-0.143, a4=0.563):
    # A-T pair double-well potential in Hartrees (x is in Bohr)
    return f*(2*a2/x0**2 + 6*a3*x/x0**3 + 12*a4*x**2/x0**4)

proton_mass = get_proton_mass()
x0      = 1.9592
N_steps = 3000

# frequency corresponding to the right well (in atomic units)
omega = np.sqrt(get_doublewell_potential_second_deriv(x0)/proton_mass)
AT_dyn_obj  = QubitDynamicsCS(n_basis=128, xo=1.5*x0, mass=proton_mass, omega=omega)

AT_dyn_obj.set_coordinate_operators(x_min=-4.0, x_max=4.0)
AT_dyn_obj.initialize_operators()
AT_dyn_obj.set_initial_state(wfn_omega=omega)

total_time = convert_fs_to_au(30.0)
AT_dyn_obj.set_propagation_time(total_time, N_steps)
AT_dyn_obj.set_hamiltonian(potential_type='quartic')

AT_dyn_obj.propagate_SOFT()
AT_dyn_obj.propagate_qt()

# Instantiate a backend object:
backend = Aer.get_backend('statevector_simulator')
AT_dyn_obj.propagate_qSOFT(backend=backend) # always shows progress bar

In [ ]:
# Checking the potential and initial coherent state:
fig, ax = plt.subplots(dpi=300)
ax.plot(AT_dyn_obj.x_grid, AT_dyn_obj._PE_grid  - get_doublewell_potential(x0) - omega/2, '-',color='black',label='A-T Basepair Potential')
ax.plot(AT_dyn_obj.x_grid, np.real(0.04*AT_dyn_obj.psio_grid.conj()*AT_dyn_obj.psio_grid),'--',color='red',label='Initial Coherent State')
ax.axhline(0, lw=0.5, color='black', alpha=1.0)
ax.axvline(-AT_dyn_obj.x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(AT_dyn_obj.x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(AT_dyn_obj.x0*1.5, lw=0.5, color='red', alpha=0.5)
ax.set_xlabel(r'$x$, Bohr',fontsize=18)
ax.set_ylabel('Energy, Hartrees',fontsize=18)
ax.tick_params(labelsize=16, grid_alpha=0.5)
plt.ylim(-0.03,0.07)
plt.legend(fontsize=14,loc='upper center')
plt.show()

In [ ]:
# Plotting the states
from scipy.interpolate import interp1d

x_1024 = np.linspace(AT_dyn_obj.x_grid[0], AT_dyn_obj.x_grid[-1], AT_dyn_obj.n_basis)
f_interp = interp1d(AT_dyn_obj.x_grid, AT_dyn_obj.dynamics_results_grid[-1], kind='cubic')
rho_interp = f_interp(x_1024)

fig, ax = plt.subplots()
ax.plot(AT_dyn_obj.x_grid, AT_dyn_obj._PE_grid  - get_doublewell_potential(x0) - omega/2, '-',color='black',label='A-T Basepair Potential')
ax.plot(AT_dyn_obj.x_grid, 0.04*np.real(AT_dyn_obj.psio_grid.conj()*AT_dyn_obj.psio_grid),'--',color='red',label='Initial Coherent State')
ax.plot(x_1024, 0.04*np.real(rho_interp.conj()*rho_interp),'-',color='blue',label=f'State at t = {convert_au_to_fs(total_time)} fs')
ax.axhline(0, lw=0.5, color='black', alpha=1.0)
ax.axvline(-x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(x0*1.5, lw=0.5, color='red', alpha=0.5)
ax.set_xlabel('$x$, Bohr',fontsize=18)
ax.set_ylabel('Energy, Hartrees',fontsize=18)
ax.tick_params(labelsize=16, grid_alpha=0.5)
plt.ylim(-0.03,0.07)
plt.legend(fontsize=14,loc='upper center')
plt.show()

In [ ]:
from qflux.closed_systems.utils import calculate_expectation_values
exp_x_grid = calculate_expectation_values(AT_dyn_obj.dynamics_results_grid, AT_dyn_obj.x_grid)


In [ ]:
# Plot expectation value of the position <X>
fig, ax = plt.subplots(1, 1, figsize=(9, 6.5))
plt.plot(AT_dyn_obj.tlist, exp_x_grid, label=r'$\left\langle x \right\rangle$ (SOFT)',
         lw=0, marker='x', markevery=20, color='dodgerblue', ms=8)
# plt.ylim(-1.55, 1.55)
ax.axhline(-AT_dyn_obj.x0, ls='--', lw=0.5, color='black', alpha=0.5)
ax.axhline(AT_dyn_obj.x0, ls='--', lw=0.5, color='black', alpha=0.5)
ax.annotate(r'\textbf{Reactant Well}', xy = (-90, 3.1),
            xycoords='data', fontsize=16, weight='bold')
ax.annotate(r'\textbf{Product Well}', xy = (1000, -2.25),
            xycoords='data', fontsize=16, weight='bold')
# plt.text(20, test.x0-0.1, 'Reactant well', fontsize = 12, backgroundcolor='white', )
# plt.text(5, -test.x0-0.1, 'Product well', fontsize = 12, backgroundcolor='white')

plt.ylim(-2.5, 3.5)
plt.xlim(-100, max(AT_dyn_obj.tlist) + 100)
plt.ylabel(r'$\left\langle x \right\rangle$', fontsize=18)
plt.xlabel(r'Time', fontsize=18)
# plt.legend(ncols=2, loc='upper center')
# plt.xlim(min(test.tlist), max(test.tlist))

## Hamiltonian Simulation [4.14]

In [ ]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.visualization import plot_histogram
from qflux import hamiltonian_simulation

# Initialize circuit
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)

# Define Hamiltonian: H = 0.5 * ZZ + 0.3 * YY
hamiltonian = {
    "ZZ": 0.5,
    "YY": 0.3,
}
# Build the Hamiltonian evolution circuit
t = np.pi / 4  # evolution time
trotter_steps = 1
U = hamiltonian_simulation(hamiltonian, quantum_register=qr, t=t, trotter_number=trotter_steps)

# Initialize a uniform superposition, add propagator and measurements
qc.h(qr)
qc.append(U,qr)
qc.measure_all()

# Simulate circuit
sim = Aer.get_backend("aer_simulator")
qobj = transpile(qc, sim)
result = sim.run(qobj).result()
counts = result.get_counts()
# Plot the result
print("Measurement counts:", counts)

# Draw circuit
qc.decompose().draw()

## QFlux Statevector Simulation for Spin Chain Dynamics [4.24]

In [ ]:
from qflux.closed_systems.spin_dynamics_oo import SpinDynamicsS

num_q = 3
evolution_timestep = 0.1
n_trotter_steps = 1
hamiltonian_coefficients = [[0.75 / 2, 0.75 / 2, 0.0, 0.65]] + [
    [0.5, 0.5, 0.0, 1.0] for _ in range(num_q - 1)
]
initial_state = "011"  # Specify the initial state as a binary string

csimulation = SpinDynamicsS(
    num_q,
    evolution_timestep,
    n_trotter_steps,
    hamiltonian_coefficients,
)
csimulation.run_dynamics(nsteps=250, state_string=initial_state)
csimulation.save_results(f"{num_q}_spin_chain")
plt.rcParams['lines.markersize']    = 4
csimulation.plot_results(f"{num_q}_spin_chain_statevector")

## QFlux Simulation for Spin Chain using Hadamard Test [4.42]

**WARNING**: This cell takes ~30 minutes to execute on Google Colab - grab a coffee!

In [ ]:
from qflux.closed_systems.spin_dynamics_oo import SpinDynamicsH

num_q = 3
evolution_timestep = 0.1
n_trotter_steps = 1
hamiltonian_coefficients = [[0.75 / 2, 0.75 / 2, 0.0, 0.65]] + [
[0.5, 0.5, 0.0, 1.0] for _ in range(num_q - 1)
]
initial_state = "011"  # Specify the initial state as a binary string

qsimulation = SpinDynamicsH(
    num_q,
    evolution_timestep,
    n_trotter_steps,
    hamiltonian_coefficients,
)
qsimulation.run_simulation(state_string=initial_state, total_time=25, num_shots=100)
qsimulation.save_results('hadamard_test')
qsimulation.plot_results('hadamard_test')

In [ ]:
qsimulation.save_results('hadamard_test')
qsimulation.plot_results('hadamard_test')

## Variation Quantum Real Time Evolution with QFlux [6.1]

In [ ]:
# --- Imports and Setups ---
'''
Modules:
- VarQRTE: Variational algorithm to simulate real-time dynamics.
- SparsePauliOp: Efficient representation of Hamiltonians.
'''
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qflux.closed_systems.VarQTE import VarQRTE, Construct_Ansatz

# --- Define the Hamiltonian ---
H = SparsePauliOp.from_list([("X", 1.0)])

# --- Define the Initial State ---
qc = QuantumCircuit(1)
qc.x(0) # Creates |1> state

# --- Perform Variational Real-Time Evolution ---
layers = 1
total_time = 12
timestep = 0.1
params = VarQRTE(layers, H, total_time, timestep, init_circ=qc)
# Params now holds the parameter values for the ansatz at each timestep

# --- Get the Expectation Values of an Observable ---
estimator = Estimator()
observable = SparsePauliOp.from_list([("Z", 1.0)])
spin1_values = []
spin2_values = []
for i in range(len(params)):
    ansatz = Construct_Ansatz(qc, params[i], H.num_qubits)
    result = estimator.run([(ansatz, observable)]).result()
    spin1_values.append(result[0].data.evs)

# --- Plot Expectation Values Over Time ---
plt.title("Spin Expectation Value Over Time")
plt.plot([i*timestep for i in range(int(total_time/timestep)+1)], spin1_values)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.show()


## Variation Quantum Imaginary Time Evolution with QFlux [6.2]

In [ ]:
# --- Imports and Setups ---
'''
Modules:
- VarQITE: Variational algorithm to simulate imaginary-time dynamics.
- SparsePauliOp: Efficient representation of Hamiltonians.
'''
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

from qflux.closed_systems.VarQTE import VarQITE, ansatz_energy

# --- Define the Hamiltonian ---
H = SparsePauliOp.from_list([("IIZ", 1.0), ("IZI", 1.0), ("ZII", 0.65), ("IXX", 1.0), ("IYY", 1.0), ("XXI", 0.75), ("YYI", 0.75)])

# --- Define the Initial State ---
qc = QuantumCircuit(3)
qc.rx(0.5, 0)
qc.rx(0.5, 1)
qc.rx(0.5, 2)

# --- Perform Variational Real-Time Evolution ---
layers = 0
total_time = 10
timestep = 0.1
params = VarQITE(layers, H, total_time, timestep, init_circ=qc)
# Params now holds the parameter values for the ansatz at each timestep

# --- Get the Expectation Value of the Energy ---
all_energies = []
for i in range(len(params)):
    print(f"Timestep {i} Energy: {ansatz_energy(qc, params[i], H)}")
    all_energies.append(ansatz_energy(qc, params[i], H)[0])

# --- Plot Expectation Values Over Time ---
plt.title("VarQITE Energy Over Imaginary Time")
plt.plot([i*timestep for i in range(int(total_time/timestep)+1)], all_energies)
plt.xlabel("Imaginary Time")
plt.ylabel("Energy (eV)")
plt.show()

## Code for running things with a realistic noise model

```python

from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

service = QiskitRuntimeService()

# Specify a QPU to use for the noise model
real_backend = service.backend("ibm_brisbane")
aer = AerSimulator.from_backend(real_backend)

# Run the sampler job locally using AerSimulator.
pm = generate_preset_pass_manager(backend=aer, optimization_level=1)
isa_qc = pm.run(qubit_testbed.quantum_circuit)
sampler = Sampler(mode=aer)
result = sampler.run([qubit_testbed.quantum_circuit]).result()
```